__Задание:__Попробуйте улучшить работу нейронной сети (разобранную на уроке), обучавшейся на датасет Fashion-MNIST <br>

Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [87]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam, SGD

In [88]:
tf.config.list_physical_devices('GPU')

[]

In [89]:
tf.random.set_seed(1) # чтобы веса инициировались одинаково

__Загрузка и обработка данных:__

In [90]:
boston = load_boston()
boston.keys()


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])

In [91]:
data = boston['data'] # тут хранится двумерный список с данными недвижки
target = boston['target'] # тут хранится одномерный список с данными о ценах
feature_names = boston["feature_names"]  #названия колонок из массива datа. см. boston.keys()
X = pd.DataFrame(data, columns=feature_names)
y = pd.DataFrame(target, columns=["price"])

X.shape

(506, 13)

In [92]:
X.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [93]:
X = StandardScaler().fit(X).transform(X)

In [94]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

__Пишу цикл для тестирования качества нейросети в зависимости от типа оптимизатора, активационной функции, числа скрытых слоев в сети, числа нейронов в каждом слое:__

In [103]:
###########################3

result = [None, 1000]

optimizers = ['Adam']
act_function = ['relu', 'sigmoid', 'tanh']
layers_num = [1,3,5,10]
neuron_num = [8, 32, 64, 128]

for opti in optimizers:
  for activator in act_function:
    for n_layer in layers_num:
      for n in neuron_num:

        model2 = Sequential()
        model2.add(Flatten(input_shape=(13, )))

        for i in range(n_layer):
          model2.add(Dense(n, activation=activator))

        model2.add(Dense(1))

        model2.compile(optimizer=opti,
              loss='mse',
              metrics=['mse'])
        
        history = model2.fit(X_train, y_train,
              epochs=10,
              batch_size=5,
              validation_data=(X_test, y_test), 
              verbose=0)
        
        train_loss, train_m = model2.evaluate(X_train, y_train, verbose=2)
        test_loss, train_m = model2.evaluate(X_test, y_test, verbose=2)

        if train_loss < result[1]:
          result[0] = f'Модель с {n_layer} слоями, {n} неронами в каждом слое, активацией {activator} и оптимизатором {opti} выдает лучшиц результат на тесте: {test_loss}'
          result[1] = train_loss




12/12 - 0s - loss: 235.7707 - mse: 235.7707 - 29ms/epoch - 2ms/step
5/5 - 0s - loss: 243.2383 - mse: 243.2383 - 23ms/epoch - 5ms/step
12/12 - 0s - loss: 36.3012 - mse: 36.3012 - 36ms/epoch - 3ms/step
5/5 - 0s - loss: 50.9146 - mse: 50.9146 - 26ms/epoch - 5ms/step
12/12 - 0s - loss: 23.9228 - mse: 23.9228 - 29ms/epoch - 2ms/step
5/5 - 0s - loss: 38.3158 - mse: 38.3158 - 21ms/epoch - 4ms/step
12/12 - 0s - loss: 20.1808 - mse: 20.1808 - 30ms/epoch - 2ms/step
5/5 - 0s - loss: 33.1470 - mse: 33.1470 - 23ms/epoch - 5ms/step
12/12 - 0s - loss: 32.4314 - mse: 32.4314 - 27ms/epoch - 2ms/step
5/5 - 0s - loss: 41.6679 - mse: 41.6679 - 22ms/epoch - 4ms/step
12/12 - 0s - loss: 11.1702 - mse: 11.1702 - 28ms/epoch - 2ms/step
5/5 - 0s - loss: 22.7770 - mse: 22.7770 - 21ms/epoch - 4ms/step
12/12 - 0s - loss: 7.4377 - mse: 7.4377 - 35ms/epoch - 3ms/step
5/5 - 0s - loss: 17.1835 - mse: 17.1835 - 23ms/epoch - 5ms/step
12/12 - 0s - loss: 7.3461 - mse: 7.3461 - 30ms/epoch - 2ms/step
5/5 - 0s - loss: 16.8528

In [104]:
result

['Модель с 10 слоями, 128 неронами в каждом слое, активацией relu и оптимизатором Adam выдает лучшиц результат на тесте: 14.5031156539917',
 5.634233474731445]

Соберу эту финальную модель ручным способом:

In [106]:
inputs = Input(shape=(13,)) # задаем размерность поступающих данных из X.shape
x_1 = Flatten()(inputs)
x_2 = Dense(128, activation='relu')(x_1)
x_3 = Dense(128, activation='relu')(x_2)
x_4 = Dense(128, activation='relu')(x_3)
x_5 = Dense(128, activation='relu')(x_4)
x_6 = Dense(128, activation='relu')(x_5)
x_7 = Dense(128, activation='relu')(x_6)
x_8 = Dense(128, activation='relu')(x_7)
x_9 = Dense(128, activation='relu')(x_8)
x_10 = Dense(128, activation='relu')(x_9)
x_11 = Dense(128, activation='relu')(x_10)
outputs = Dense(1)(x_11)                         # не понял почему тут 1 и только один аргумент

model = Model(inputs, outputs)

model.compile(optimizer='Adam',
              loss='mse',
              metrics=['mse'])

#model.summary()

history = model.fit(X_train, y_train,
              epochs=10,
              batch_size=5,
              validation_data=(X_test, y_test), 
              verbose=0)

train_loss, train_m = model.evaluate(X_train, y_train, verbose=2)
test_loss, test_m = model.evaluate(X_test, y_test, verbose=2)
train_loss, test_loss

12/12 - 0s - loss: 4.7353 - mse: 4.7353 - 33ms/epoch - 3ms/step
5/5 - 0s - loss: 14.8090 - mse: 14.8090 - 24ms/epoch - 5ms/step


(4.735265254974365, 14.808952331542969)

Соберем финальную модель другим способом:

In [107]:
model_03 = Sequential([
    Flatten(input_shape=(13, )), 
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1)
])

model_03.compile(optimizer='Adam',
               loss='mse',
               metrics=['mse'])

history = model_03.fit(X_train, y_train,
              epochs=10,
              batch_size=5,
              validation_data=(X_test, y_test), 
              verbose=0)

train_loss, train_m = model_03.evaluate(X_train, y_train, verbose=2)
test_loss, test_m = model_03.evaluate(X_test, y_test, verbose=2)
train_loss, test_loss

12/12 - 0s - loss: 3.9896 - mse: 3.9896 - 34ms/epoch - 3ms/step
5/5 - 0s - loss: 13.5117 - mse: 13.5117 - 25ms/epoch - 5ms/step


(3.989556074142456, 13.51174259185791)

Здесь мсе на тесте даже лучше.

Ну и посмотрим совпадут ли значения мсе из цикла лучшей модели и этой:

In [109]:
model4 = Sequential()

model4.add(Flatten(input_shape=(13, )))

model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))

model4.add(Dense(1))

model4.compile(optimizer='Adam',
               loss='mse',
               metrics=['mse'])

history4 = model4.fit(X_train, y_train,
              epochs=10,
              batch_size=5,
              validation_data=(X_test, y_test), 
              verbose=0)

train_loss, train_m = model4.evaluate(X_train, y_train, verbose=2)
test_loss, test_m = model4.evaluate(X_test, y_test, verbose=2)
train_loss, test_loss


12/12 - 0s - loss: 4.8398 - mse: 4.8398 - 34ms/epoch - 3ms/step
5/5 - 0s - loss: 13.7261 - mse: 13.7261 - 24ms/epoch - 5ms/step


(4.839778423309326, 13.726066589355469)